# 데이터 불러오기

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install fastparquet

import pandas as pd
import numpy as np
import datetime as dt

path1 = '/content/drive/MyDrive/MyCP2/ItemsUsers.parquet'
path2 = '/content/drive/MyDrive/MyCP2/Items.parquet'
path3 = '/content/drive/MyDrive/MyCP2/Users.parquet'

data1 = pd.read_parquet(path1, engine='fastparquet')
data2 = pd.read_parquet(path2, engine='fastparquet')
data3 = pd.read_parquet(path3, engine='fastparquet')

df = data1.copy()
df_items = data2.copy()
df_users = data3.copy()

# 추가 EDA

In [ ]:
## items_features, users_features 컬럼 수정

# items_features: (items_id_num, [category])
items = df_items[['category']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
items = items.str.split(',')
items = list(zip(df_items['items_id_num'], items))

df_items['items'] = items

# users_features: (users_id_num, [events])
users = df_users[['events']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
users = users.str.split(',')
users = list(zip(df_users['users_id_num'], users))

df_users['users'] = users

# 기존 features 컬럼 삭제
df_users.drop(['users_features'], axis=1, inplace=True)
df_items.drop(['items_features'], axis=1, inplace=True)

# 새 features 컬럼 이름 수정
df_users.rename(columns={'users':'users_features'}, inplace=True)
df_items.rename(columns={'items':'items_features'}, inplace=True)

In [ ]:
# 수정한 파일이 파케이로 저장이 안 되어서 피클로 저장
path='/content/drive/MyDrive/MyCP2/'

df_items.to_pickle(path + 'df_items.pkl')
df_users.to_pickle(path + 'df_users.pkl')

# 데이터 불러오기2

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !pip install fastparquet

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

path1 = '/content/drive/MyDrive/MyCP2/ItemsUsers.parquet'
path2 = '/content/drive/MyDrive/MyCP2/df_items.pkl'
path3 = '/content/drive/MyDrive/MyCP2/df_users.pkl'

df = pd.read_parquet(path1, engine='fastparquet')
df_items = pd.read_pickle(path2)
df_users = pd.read_pickle(path3)

# 데이터 확인

In [ ]:
df.head(1)

,event_time,event_type,product_id,category_id,brand,price,user_id,main_category,sub_category,sub_sub_category,users_id_num,items_id_num,items_users_id_tuple
0,2019-10-02 00:00:00+04:00,view,6200670,-813694761,luxell,39.369999,516364071,appliances,environment,air_heater,0,0,"[0, 0]"


In [ ]:
df_items.head(1)

,product_id,view_num,cart_num,purchase_num,items_id_num,category,items_features
0,6200670,279,0,2,0,"appliances,environment,air_heater","(0, [appliances, environment, air_heater])"


In [ ]:
df_users.head(1)

,user_id,product_id,events,users_id_num,users_features
0,64078358,10600284,view,0,"(0, [view])"


# Baseline

# BestSeller

In [ ]:
def bestseller(n_items):
    df_pur = df_items[['product_id', 'purchase_num']]
    df_pur.set_index('product_id')
    items_sort = df_pur.sort_values(by='purchase_num', ascending=False)[:n_items]
    recom_items = df_items.loc[items_sort.index]
    recommendations = recom_items[['product_id', 'view_num', 'cart_num', 'purchase_num', 'category']]

    return recommendations

In [ ]:
# 베스트셀러 top10
bestseller(10)

,product_id,view_num,cart_num,purchase_num,category
205,4804056,174185,22222,12027,"electronics,audio,headphone"
51,5100816,159732,17011,7054,"NaN,NaN,NaN"
128,4804295,102301,7710,3452,"electronics,audio,headphone"
676,4804055,73648,5627,3286,"electronics,audio,headphone"
185,1801690,72926,5886,2684,"electronics,video,tv"
61,12703015,54026,1287,1650,"NaN,NaN,NaN"
227,3700926,45260,3342,1636,"appliances,environment,vacuum"
37,1801766,70153,2544,1522,"electronics,video,tv"
45,3600661,61126,2249,1451,"appliances,kitchen,washer"
486,4803977,40981,2695,1403,"electronics,audio,headphone"


In [ ]:
from tqdm import tqdm

In [ ]:
def precision(n_samples):
    K = 10
    pre = [] # 사용자별 precision
    for user in tqdm(user_list[:n_samples]):
        a = df_users.loc[(df_users['user_id']==user) &
                     (df_users['product_id'].isin(best)) &
                     (df_users['events'].str.contains('purchase')), "product_id"].nunique()
        pre.append(a / K)

    return np.mean(pre)

- 구매 이력이 없는 애들은 평가가 불가능하다. 과거 (취향) 걍 냅다 베스트셀러를 추천한다.  
- 구매 이력이 있는 애들은 베이스라인을 베스트셀러로 잡고 lightfm 적용해서 precision 비교한다

In [ ]:
precision(1000)

100%|██████████| 1000/1000 [1:23:34<00:00,  5.01s/it]


0.0

- 이거는 정확도 측정을 못해서 baseline 으로 쓸 수 없나?

In [ ]:
k = 20

# training data에 user가 없는 경우엔 추천이 안되므로 인기도 기반 추천 리스트를 미리 만들어 놓는다.
popular_item_at_k = list(train_df.groupby('item_id')['Quantity'].sum().sort_values(ascending=False)[:k].index)

precisions = []
recalls = []

# 사용자 별로 Top N 추천을 하여 사용자 별 precision recall을 구한다.

cold_start = 0
user_grouped = test_df.groupby('user_id')

for user_id, user_df in user_grouped:

    # test 데이터에 존재하는 사용자가 이미 구매한 아이템 = 정답
    buy_items = [stock_code_map[stock_code] if stock_code in stock_code_map else stock_code \
                 for stock_code in user_df['StockCode'].unique()]

    if customer_id in customer_id_map:
        user_id = customer_id_map[customer_id]
        user_vector = model.user_factors[user_id]
        scores = item_vecs.dot(user_vector)
        top_k_item = np.argpartition(scores, -k)[-k:]

        # 모델에서 top k 아이템을 선정하는 함수를 이미 제공함
        # top_k_item = [item for item, _ in model1.recommend(user_id, sparse_user_item.T, k, filter_already_liked_items=False)]

    else:
        cold_start += 1
        top_k_item = popular_item_at_k

    intersect = set(buy_items).intersection(set(top_k_item))

    precision = len(intersect)/len(top_k_item)
    recall = len(intersect)/len(buy_items)

    precisions.append(precision)
    recalls.append(recall)
print('\n')
print('precision@k:', np.mean(np.asarray(precisions)))
print('recall@k:', np.mean(np.asarray(recalls)))

# LightFM Model

In [3]:
# !pip install lightfm

In [ ]:
# import lightfm
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k


import warnings
warnings.filterwarnings('ignore')

In [ ]:
## generating features list for mapping ##

# items_features_list
i_features = df_items[['category']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
i_features = i_features.str.split(',')
i_features = i_features.apply(pd.Series).stack().reset_index(drop=True)

items_features_list = i_features

# users_features_list
u_features = df_users[['events']].apply(
    lambda x: ','.join(x.map(str)), axis=1)
u_features = u_features.str.split(',')
u_features = u_features.apply(pd.Series).stack().reset_index(drop=True)

users_features_list = u_features

In [ ]:
## save items/users_features_list ##
# df_items_features = pd.DataFrame(items_features_list)
# df_users_features = pd.DataFrame(users_features_list)

# df_items_features.rename(columns={0:'features_list'}, inplace=True)
# df_users_features.rename(columns={0:'features_list'}, inplace=True)

# df_items_features.to_parquet('/content/drive/MyDrive/MyCP2/df_items_features.parquet')
# df_users_features.to_parquet('/content/drive/MyDrive/MyCP2/df_users_features.parquet')

In [ ]:
path4 = '/content/drive/MyDrive/MyCP2/df_items_features.parquet'
path5 = '/content/drive/MyDrive/MyCP2/df_users_features.parquet'

df_items_features = pd.read_parquet(path4, engine='fastparquet')
df_users_features = pd.read_parquet(path5, engine='fastparquet')

users_features_list = pd.Series(df_users_features['features_list'])
items_features_list = pd.Series(df_items_features['features_list'])

In [ ]:
# users_features_list

0           view
1           view
2           view
3           view
4           view
            ... 
14363478    view
14363479    view
14363480    view
14363481    view
14363482    view
Name: features_list, Length: 14363483, dtype: object

In [ ]:
# items_features_list

0          appliances
1         environment
2          air_heater
3                 NaN
4                 NaN
             ...     
396979            NaN
396980            NaN
396981            NaN
396982            NaN
396983            NaN
Name: features_list, Length: 396984, dtype: object

In [ ]:
# create an instance of Dataset which holds the interaction matrix
dataset = Dataset()

# create the user/item id mappings
# fit(users, items, user_features=None, item_features=None)
dataset.fit(
    set(df_users['users_id_num']),
    set(df_items['items_id_num']),
    user_features=users_features_list,
    item_features=items_features_list) # 여기서 중요한 것은, 이 때 argument로 들어가는 객체에 결측값은 없어야 한다


# build interactions
# returns 2 COO sparse matrices, interations and weights
interactions, weights = dataset.build_interactions(df['items_users_id_tuple'])

# build item/user features
items_features = dataset.build_item_features(df_items['items_features'])
users_features = dataset.build_user_features(df_users['users_features'])

In [ ]:
# interactions

<2042292x132328 sparse matrix of type '<class 'numpy.int32'>'
	with 24358206 stored elements in COOrdinate format>

In [ ]:
# users_features

<2042292x2042295 sparse matrix of type '<class 'numpy.float32'>'
	with 4404168 stored elements in Compressed Sparse Row format>

In [ ]:
# items_features

<132328x132436 sparse matrix of type '<class 'numpy.float32'>'
	with 342928 stored elements in Compressed Sparse Row format>

In [ ]:
df.head(1)

,event_time,event_type,product_id,category_id,brand,price,user_id,main_category,sub_category,sub_sub_category,users_id_num,items_id_num,items_users_id_tuple
0,2019-10-02 00:00:00+04:00,view,6200670,-813694761,luxell,39.369999,516364071,appliances,environment,air_heater,0,0,"[0, 0]"


In [ ]:
# the number of unique users/items
num_users, num_items = dataset.interactions_shape()
print(f'Num users: {num_users}, num_items: {num_items}')

Num users: 2042292, num_items: 132328


# Train, Test Set Split

- 보통 과거 데이터를 가지고 미래 데이터를 예측하기 때문에, 특정한 시간을 기준으로 전후 데이터를 train / test 데이터로 나눈다.
- 특히 커머스 데이터와 같은 경우, 같은 상품을 다시 구매할 수 있기 때문에 이미 구매한 상품을 다시 추천하기도 한다.
- 하지만 여기서는 시간을 고려하지 않고 랜덤하게 학습, 테스트 셋을 추출한다.
    - 시계열이 아닌 이유:
        1. user별 view, cart, purchase 여부를 확인하고 한 행으로 묶음
    - product는 전체 기간에 대한 총 view, cart, purchase 수 컬럼을 만들었지만 모델링에 포함되지 않는 피쳐이므로 괜찮다.  
    근데 베스트셀러 추천에는 데이터 누수 문제가 있음

In [ ]:
# split train, test set
train, test = cross_validation.random_train_test_split(
    interactions, test_percentage=0.25,
    random_state=42)

# check shape
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

Shape of train interactions: (2042292, 132328)
Shape of test interactions: (2042292, 132328)


In [ ]:
model_train = LightFM(
    no_components=30,  # latent factor (잠재요인) 을 몇개로 할거냐?
    learning_rate=0.05,
    loss='warp',
    random_state=42)

model_train.fit(
    interactions=train,
    item_features=items_features,
    user_features=users_features,
    epochs=5,
    verbose=True)

Epoch: 100%|██████████| 5/5 [10:29<00:00, 125.99s/it]


# 모델 저장

In [ ]:
# # save the model
# import pickle

# with open('/content/drive/MyDrive/MyCP2/LightFM_temp.pkl','wb') as fw:
#     pickle.dump(model, fw)

# # load the model
# with open('/content/drive/MyDrive/MyCP2/LightFM_temp.pkl', 'rb') as f:
#     model = pickle.load(f)

# LightFM Evaluation

In [ ]:
K = 10

precision_lfm_test = lightfm_prec_at_k(model_train, test, k=K, item_features=items_features, user_features=users_features).mean()

In [ ]:
precision_lfm_test

0.014344113

# LightFM Recommendation System

In [ ]:
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)

In [ ]:
def recommend_product(user_id):

    for user in user_id:

        # user가 이전에 샀던 item의 product_id
        if (df['user_id'] == user) & (df['event_type'] == 'purchase'):
            purchased_items_id = df.loc[(df['user_id'] == user) & (df['event_type'] == 'purchase')][:3]['product_id']
        elif (df['user_id'] == user) & (df['event_type'] == 'cart'):
            purchased_items_id = df.loc[(df['user_id'] == user) & (df['event_type'] == 'cart')][:3]['product_id']
        else:
            purchased_items_id = df.loc[(df['user_id'] == user) & (df['event_type'] == 'view')][:3]['product_id']

        # purchased_items_id = df.loc[(df['user_id'] == user) & (df['event_type'] == 'purchase')][:3]['product_id']
        df_items_purchased = df_items.loc[df_items['product_id'].isin(purchased_items_id)]

        print('Items purchased by user id:(' + str(user) + ')')

        display_side_by_side(
            df_items_purchased[['product_id', 'category']],
            df_users.loc[df_users['user_id'] == user][['user_id', 'product_id', 'events']])

        # predict
        discard_items_id = df_items_purchased['items_id_num'].values.tolist()
        df_use_for_prediction = df_items.loc[~df_items['items_id_num'].isin(discard_items_id)]
        items_id_for_prediction = df_use_for_prediction['items_id_num'].values.tolist()

        # score
        user_id_num = df.loc[df['user_id'] == user]['users_id_num'].values[0].tolist()

        scores = model_train.predict(
            user_id_num,
            items_id_for_prediction,
            item_features=items_features,
            user_features=users_features)

        df_use_for_prediction['scores'] = scores
        df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:5]

        print('Recommended products for user id:(' + str(user) + ')')

        display(df_use_for_prediction[['product_id', 'category']])

In [ ]:
recommend_product([566232270, 566236063])

Items purchased by user id:(566232270)


,product_id,category
185,1801690,"electronics,video,tv"
,user_id,events
13924181,566232270,"view,cart,purchase"


Recommended products for user id:(566232270)


,product_id,category
205,4804056,"electronics,audio,headphone"
51,5100816,"NaN,NaN,NaN"
128,4804295,"electronics,audio,headphone"
676,4804055,"electronics,audio,headphone"
37,1801766,"electronics,video,tv"


Items purchased by user id:(566236063)


,product_id,category
1261,3700790,"appliances,environment,vacuum"
2834,3700823,"appliances,environment,vacuum"
4314,10800172,"NaN,NaN,NaN"
,user_id,events
13924358,566236063,"view,cart,purchase"
13924359,566236063,view
13924360,566236063,view
13924361,566236063,view


Recommended products for user id:(566236063)


,product_id,category
45,3600661,"appliances,kitchen,washer"
227,3700926,"appliances,environment,vacuum"
205,4804056,"electronics,audio,headphone"
487,3600163,"appliances,kitchen,washer"
92,3600666,"appliances,kitchen,washer"


In [ ]:
# 카트이력 사용자
recommend_product([566235051, 371460797])

Items purchased by user id:(566235051)


product_id 
 category 
 
 
 
 
 
 
 
 
 user_id 
 product_id 
 events 
 
 
 
 
 13924293 
 566235051 
 4804295 
 view,cart

Recommended products for user id:(566235051)


,product_id,category
51,5100816,"NaN,NaN,NaN"
205,4804056,"electronics,audio,headphone"
185,1801690,"electronics,video,tv"
37,1801766,"electronics,video,tv"
128,4804295,"electronics,audio,headphone"


Items purchased by user id:(371460797)


product_id 
 category 
 
 
 
 
 
 
 
 
 user_id 
 product_id 
 events 
 
 
 
 
 1199 
 371460797 
 3700551 
 view 
 
 
 1200 
 371460797 
 3701016 
 view,cart 
 
 
 1201 
 371460797 
 3701025 
 view 
 
 
 1202 
 371460797 
 3701154 
 view 
 
 
 1203 
 371460797 
 4400404 
 view 
 
 
 1204 
 371460797 
 4400468 
 view 
 
 
 1205 
 371460797 
 7100893 
 view 
 
 
 1206 
 371460797 
 25900025 
 view

Recommended products for user id:(371460797)


,product_id,category
45,3600661,"appliances,kitchen,washer"
205,4804056,"electronics,audio,headphone"
227,3700926,"appliances,environment,vacuum"
487,3600163,"appliances,kitchen,washer"
92,3600666,"appliances,kitchen,washer"
